1. How to run an elastic traffic assignment on a real network

In [2]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import sys
sys.path.append("../../..")
import numpy as np
import pandas as pd
import geopandas as gpd
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network
from pyproj import Proj, transform
from pickle import dump

In [3]:
# Creating the network for Brussels. 
# fill in these parameters 
# IMPORTANT: Use the same parameter values for the buffer as was done in STA_prep_script!
ext = 0  # 0 or 2
buffer = 40
city = 'BRUSSEL'

HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map'
zoning_path = data_path  + os.path.sep + 'QGIS' + os.path.sep + city + "_" + str(buffer) + "_10_aggr_comb.shp"
if ext == 2:
    od_path = data_path + os.path.sep + 'STA' + os.path.sep + 'OD_matrix' + os.path.sep + city + "_ext_tr_" + str(buffer) + "_9_10.xlsx"
else: 
    od_path = data_path + os.path.sep + 'STA' + os.path.sep + 'OD_matrix' + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"
print(zoning_path)

C:\Users\silky\Documents\School\KU Leuven\Master\YEAR 2\IP2\IP2\toll_optimization\case_studies\Brussel_elastic_demand\..\..\data_map\QGIS\BRUSSEL_40_10_aggr_comb.shp


In [4]:
# Creating the network for Brussels. 
network = road_network_from_place("Brussels", buffer_dist_close=buffer*1000)
network = relabel_graph(network)
show_network(network,notebook=True)

In [ ]:
zoning = gpd.read_file(zoning_path)
od = pd.read_excel(od_path)
old_od = od.to_numpy() # The OD matrix is now stored in a numpy array

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
network = add_centroids(network, x_centroids,y_centroids,k=1, method='link')
network = relabel_graph(network)
show_network(network, notebook=True)

# Create OD graph and plot demand
old_graph = od_graph_from_matrix(old_od,x_centroids,y_centroids) 
# demand plot is useless since there are too many OD-pairs 

In [ ]:
%store -r zone_elasticity_matrix

# A matrix
A = 1.5
# B matrix
B = zone_elasticity_matrix
print(B)

[[0.384 0.384 0.384 ... 0.422 0.422 0.422]
 [0.384 0.384 0.384 ... 0.422 0.422 0.422]
 [0.384 0.384 0.384 ... 0.422 0.422 0.422]
 ...
 [0.422 0.422 0.422 ... 0.576 0.499 0.499]
 [0.422 0.422 0.422 ... 0.499 0.576 0.576]
 [0.422 0.422 0.422 ... 0.499 0.576 0.576]]


In [1]:
# First iteration
assignment = StaticAssignment(network,old_graph)
result = assignment.run('dial_b')    
new_od = (A-result.skim)/B
print(np.max(abs((new_od-old_od))))

NameError: name 'StaticAssignment' is not defined

In [ ]:
i = 1 
max_iterations = 3
max_deviation_per_od = 1
while np.max(abs((new_od - old_od))) > max_deviation_per_od and i < max_iterations:
    # Perhaps MSA stap
    old_od = new_od
    old_graph =  od_graph_from_matrix(old_od, x_centroids, y_centroids)
    assignment = StaticAssignment(network,old_graph)
    result = assignment.run('dial_b')
    new_od = (A-result.skim)/B
    i += 1
    print(np.max(abs((new_od-old_od))))

2. HEEDS preparation

In [ ]:
# Only the network with the centroids will remain static throughout the HEEDS optimization process.
# The OD is elastic, so should not be stored beforehand. 

buffer = str(buffer)
HEEDS_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map' + os.path.sep + 'HEEDS_input'
network_path = HEEDS_path + os.path.sep + 'network_with_centroids' + os.path.sep + 'elastic_' + city + '_' + buffer

with open(network_path, 'wb') as network_file:
    dump(network, network_file)
    print(f'network saved at f{network_path}')
